# Poker Round Simulator

This is a clean version of the poker simulator that works reliably in JupyterLab.

**Features:**
1. Traditional Poker Hands with joker support
2. Exotic "Illegal" Hands like Rainbow Straight and Flush Five
3. Mod Cards that modify gameplay
4. Character Classes with unique abilities
5. Multi-Round Tournament system

Let's start by setting up the core components!

In [69]:
# Import all necessary components
# pylint: disable=import-error
from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Tuple, Literal, Set, cast
import random
import enum
from itertools import combinations

print("Welcome to the Poker Round Simulator!")

Welcome to the Poker Round Simulator!


In [70]:
# Define core data structures with full type safety

DEFAULT_STARTING_CHIPS = 50
RANKS = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"]
SUITS = ["S", "H", "D", "C"]  # Using letters instead of symbols to avoid encoding issues

class Rarity(str, enum.Enum):
    Common = "Common"
    Uncommon = "Uncommon"
    Rare = "Rare"
    Epic = "Epic"

class CharacterClass(str, enum.Enum):
    Warrior = "Warrior"
    Rogue = "Rogue"
    Mage = "Mage"
    Cleric = "Cleric"
    Bard = "Bard"
    Wizard = "Wizard"
    Jester = "Jester"
    Pauper = "Pauper"
    Knight = "Knight"

class ModCardEffects(str, enum.Enum):
    Draw1 = "Draw1"
    Draw2 = "Draw2"
    Discard1 = "Discard1"
    Discard2 = "Discard2"
    Steal1 = "Steal1"
    Steal2 = "Steal2"
    SneakySwap = "SneakySwap"
    RoyalDividend = "RoyalDividend"
    GainChips = "GainChips"

class ModCardType(str, enum.Enum):
    Player = "player"
    Showdown = "showdown"
    Payout = "payout"
    Global = "global"

class ModCardTypeFlavor(str, enum.Enum):
    Pawn = "Pawn"
    Knight = "Knight"
    Rook = "Rook"
    Queen = "Queen"

class DeckType(str, enum.Enum):
    Standard = "Standard"
    Proletariat = "Proletariat"
    Fibonacci = "Fibonacci"
    Oddball = "Oddball"
    EvenSteven = "EvenSteven"
    SeeingRed = "SeeingRed"
    DarkSoul = "DarkSoul"
    Madness = "Madness"

# Card types
CardValue = str  # Format: "{rank}{suit}" or "JOKER"

# Hand types
HandType = Literal[
    "High Card", "One Pair", "Two Pair", "Three of a Kind", "Straight", 
    "Flush", "Full House", "Four of a Kind", "Straight Flush", "Royal Flush",
    "Flush Four", "Sandwich Hand", "Odd Straight", "Even Straight", 
    "Skipping Straight", "Rainbow Straight", "Flush House", "Five of a Kind", "Flush Five"
]

class ModCard(BaseModel):
    name: str
    description: str
    effect: ModCardEffects
    type: ModCardType
    flavorType: ModCardTypeFlavor
    rarity: Rarity
    burnOnUse: Optional[bool] = False

class PlayerCharacterCard(BaseModel):
    name: str
    char_class: CharacterClass
    deck_modifier: Optional[str] = None
    ability: str
    starting_chips: int = DEFAULT_STARTING_CHIPS
    description: Optional[str] = None

print("Core data structures defined with type safety!")
print(f"Available character classes: {list(CharacterClass)}")
print(f"Available mod effects: {list(ModCardEffects)}")

Core data structures defined with type safety!
Available character classes: [<CharacterClass.Warrior: 'Warrior'>, <CharacterClass.Rogue: 'Rogue'>, <CharacterClass.Mage: 'Mage'>, <CharacterClass.Cleric: 'Cleric'>, <CharacterClass.Bard: 'Bard'>, <CharacterClass.Wizard: 'Wizard'>, <CharacterClass.Jester: 'Jester'>, <CharacterClass.Pauper: 'Pauper'>, <CharacterClass.Knight: 'Knight'>]
Available mod effects: [<ModCardEffects.Draw1: 'Draw1'>, <ModCardEffects.Draw2: 'Draw2'>, <ModCardEffects.Discard1: 'Discard1'>, <ModCardEffects.Discard2: 'Discard2'>, <ModCardEffects.Steal1: 'Steal1'>, <ModCardEffects.Steal2: 'Steal2'>, <ModCardEffects.SneakySwap: 'SneakySwap'>, <ModCardEffects.RoyalDividend: 'RoyalDividend'>, <ModCardEffects.GainChips: 'GainChips'>]


In [71]:
# Create game decks

def create_real_deck() -> List[CardValue]:
    """Create a standard 52-card deck + 2 jokers."""
    
    deck = []
    for rank in RANKS:
        for suit in SUITS:
            deck.append(f"{rank}{suit}")
    
    # Add jokers
    deck.extend(["JOKER", "JOKER"])
    return deck

def create_custom_deck(type: DeckType) -> List[CardValue]:
    """Create a custom deck with specific cards."""

    match type:
        case DeckType.Proletariat:
            # Only number cards + jokers & Aces - no face cards
            deck = [f"{rank}{suit}" for rank in RANKS if rank not in ["J", "Q", "K", "A"] for suit in SUITS]
            deck.extend(["JOKER", "JOKER"])
            return deck
        case DeckType.Fibonacci:
            deck = [f"{rank}{suit}" for rank in RANKS for suit in SUITS if rank in ["A", "1", "2", "3", "5", "8", "J", "Q", "K"]]
            deck.extend(["JOKER", "JOKER"])
            return deck
        case DeckType.Oddball:
            deck = [f"{rank}{suit}" for rank in RANKS for suit in SUITS if rank in ["A", "3", "5", "7", "9", "J", "Q", "K"]]
            deck.extend(["JOKER", "JOKER"])
            return deck
        case DeckType.EvenSteven:
            deck = [f"{rank}{suit}" for rank in RANKS for suit in SUITS if rank in ["2", "4", "6", "8", "10", "J", "Q", "K"]]
            deck.extend(["JOKER", "JOKER"])
            return deck
        case DeckType.SeeingRed:
            # Only red suits + jokers
            deck = [f"{rank}{suit}" for rank in RANKS for suit in SUITS if suit in ["H", "D"]]
            deck.extend(["JOKER", "JOKER"])
            return deck
        case DeckType.DarkSoul:
            # Only black suits + jokers
            deck = [f"{rank}{suit}" for rank in RANKS for suit in SUITS if suit in ["S", "C"]]
            deck.extend(["JOKER", "JOKER"])
            return deck
        case DeckType.Madness:
            deck = create_real_deck()
            # Add extra jokers for madness
            deck.extend(["JOKER", "JOKER", "JOKER", "JOKER"])
            return deck
        case _:
            return create_real_deck()

# Create game data
real_deck = create_real_deck()
custom_decks = {
    DeckType.Madness: create_custom_deck(DeckType.Madness),
    DeckType.DarkSoul: create_custom_deck(DeckType.DarkSoul),
    DeckType.SeeingRed: create_custom_deck(DeckType.SeeingRed),
    DeckType.EvenSteven: create_custom_deck(DeckType.EvenSteven),
    DeckType.Oddball: create_custom_deck(DeckType.Oddball),
    DeckType.Fibonacci: create_custom_deck(DeckType.Fibonacci),
    DeckType.Proletariat: create_custom_deck(DeckType.Proletariat)
}

mod_deck = [
    ModCard(
        name="Extra Draw", 
        description="Draw 1 additional card", 
        effect=ModCardEffects.Draw1, 
        type=ModCardType.Player, 
        flavorType=ModCardTypeFlavor.Pawn, 
        rarity=Rarity.Common
    ),
    ModCard(
        name="Sneaky Swap", 
        description="Swap a card with opponent", 
        effect=ModCardEffects.SneakySwap, 
        type=ModCardType.Player, 
        flavorType=ModCardTypeFlavor.Pawn, 
        rarity=Rarity.Uncommon
    ),
    ModCard(
        name="Royal Dividend", 
        description="Gain +3 chips", 
        effect=ModCardEffects.RoyalDividend, 
        type=ModCardType.Payout, 
        flavorType=ModCardTypeFlavor.Queen, 
        rarity=Rarity.Uncommon
    )
]

players = [
    PlayerCharacterCard(
        name="Scruffy McMuffins", 
        char_class=CharacterClass.Rogue, 
        deck_modifier="Normal", 
        ability="Steal 1 opponent card 2x per round"
    ),
    PlayerCharacterCard(
        name="Patches", 
        char_class=CharacterClass.Jester, 
        deck_modifier="Remove face cards", 
        ability="Blind money refunded"
    ),
    PlayerCharacterCard(
        name="Sir Whiskers", 
        char_class=CharacterClass.Warrior, 
        deck_modifier="Normal", 
        ability="Reveal opponent card once per round"
    )
]

print("  - Full deck created with standard 52 cards + 2 jokers")
print(f"Real Deck: {len(real_deck)} cards")
print(f"Custom Decks: {len(custom_decks)} decks created")

print("Custom player decks created with various custom rules:")
for dtype, deck in custom_decks.items():
    print(f"  - {dtype.value}: {len(deck)} cards")
    print(f"    cards: {deck[:-1]}...")
print(f"Sample cards: {real_deck[:10]}...")
print(f"\nMod Deck: {len(mod_deck)} cards")
for mod in mod_deck:
    print(f"  - {mod.name} ({mod.rarity}): {mod.description}")
print(f"\nPlayers: {len(players)} characters")
for player in players:
    print(f"  - {player.name} ({player.char_class}): {player.ability}")

  - Full deck created with standard 52 cards + 2 jokers
Real Deck: 54 cards
Custom Decks: 7 decks created
Custom player decks created with various custom rules:
  - Madness: 58 cards
    cards: ['2S', '2H', '2D', '2C', '3S', '3H', '3D', '3C', '4S', '4H', '4D', '4C', '5S', '5H', '5D', '5C', '6S', '6H', '6D', '6C', '7S', '7H', '7D', '7C', '8S', '8H', '8D', '8C', '9S', '9H', '9D', '9C', '10S', '10H', '10D', '10C', 'JS', 'JH', 'JD', 'JC', 'QS', 'QH', 'QD', 'QC', 'KS', 'KH', 'KD', 'KC', 'AS', 'AH', 'AD', 'AC', 'JOKER', 'JOKER', 'JOKER', 'JOKER', 'JOKER']...
  - DarkSoul: 28 cards
    cards: ['2S', '2C', '3S', '3C', '4S', '4C', '5S', '5C', '6S', '6C', '7S', '7C', '8S', '8C', '9S', '9C', '10S', '10C', 'JS', 'JC', 'QS', 'QC', 'KS', 'KC', 'AS', 'AC', 'JOKER']...
  - SeeingRed: 28 cards
    cards: ['2H', '2D', '3H', '3D', '4H', '4D', '5H', '5D', '6H', '6D', '7H', '7D', '8H', '8D', '9H', '9D', '10H', '10D', 'JH', 'JD', 'QH', 'QD', 'KH', 'KD', 'AH', 'AD', 'JOKER']...
  - EvenSteven: 34 cards
    c

In [72]:
# Hand evaluation functions

def card_rank(card: CardValue) -> int:
    """Convert a card to its numeric rank."""
    if card == "JOKER":
        return 0
    
    rank_str = card[:-1]  # Everything except last character (suit)
    
    if rank_str in ["J", "Q", "K", "A"]:
        rank_map = {"J": 11, "Q": 12, "K": 13, "A": 14}
        return rank_map[rank_str]
    
    return int(rank_str)

def evaluate_standard_poker(hand: List[CardValue]) -> HandType:
    """Evaluate standard poker hands with joker support."""
    if len(hand) != 5:
        raise ValueError(f"Hand must contain exactly 5 cards, got {len(hand)}")
    
    jokers = [card for card in hand if card == "JOKER"]
    real_cards = [card for card in hand if card != "JOKER"]
    num_jokers = len(jokers)
    
    if num_jokers == 0:
        # No jokers - standard evaluation
        suits = [card[-1] for card in hand]
        ranks = [card_rank(card) for card in hand]
        rank_counts = {}
        
        for r in ranks:
            rank_counts[r] = rank_counts.get(r, 0) + 1
        
        sorted_ranks = sorted(ranks)
        is_flush = len(set(suits)) == 1
        is_straight = all(sorted_ranks[i+1] - sorted_ranks[i] == 1 for i in range(4))
        
        # Check hands in order of strength
        if is_straight and is_flush:
            if sorted_ranks == [10, 11, 12, 13, 14]:
                return cast(HandType, "Royal Flush")
            return cast(HandType, "Straight Flush")
        
        if 4 in rank_counts.values():
            return cast(HandType, "Four of a Kind")
        
        if sorted(rank_counts.values()) == [2, 3]:
            return cast(HandType, "Full House")
        
        if is_flush:
            return cast(HandType, "Flush")
        
        if is_straight:
            return cast(HandType, "Straight")
        
        if 3 in rank_counts.values():
            return cast(HandType, "Three of a Kind")
        
        pair_count = sum(1 for count in rank_counts.values() if count == 2)
        if pair_count == 2:
            return cast(HandType, "Two Pair")
        elif pair_count == 1:
            return cast(HandType, "One Pair")
        
        return cast(HandType, "High Card")
    else:
        # With jokers - simplified evaluation
        ranks = [card_rank(card) for card in real_cards]
        rank_counts = {}
        for r in ranks:
            rank_counts[r] = rank_counts.get(r, 0) + 1
        
        # Use jokers to make best possible hand
        for rank in set(ranks):
            if rank_counts[rank] + num_jokers >= 4:
                return cast(HandType, "Four of a Kind")
        
        for rank in set(ranks):
            if rank_counts[rank] + num_jokers >= 3:
                return cast(HandType, "Three of a Kind")
        
        for rank in set(ranks):
            if rank_counts[rank] + num_jokers >= 2:
                return cast(HandType, "One Pair")
        
        return cast(HandType, "High Card")

# Hand ranking order
HAND_RANK_ORDER = [
    "High Card", "One Pair", "Two Pair", "Three of a Kind", "Straight", 
    "Flush", "Full House", "Four of a Kind", "Straight Flush", "Royal Flush",
    "Flush Four", "Sandwich Hand", "Odd Straight", "Even Straight", 
    "Skipping Straight", "Rainbow Straight", "Flush House", "Five of a Kind", "Flush Five"
]

def get_hand_rank(hand_type: HandType) -> int:
    """Get numeric rank for comparison."""
    return HAND_RANK_ORDER.index(hand_type)

print("Hand evaluation functions loaded!")
print(f"Hand rankings (weakest to strongest): {HAND_RANK_ORDER[:5]}...")

Hand evaluation functions loaded!
Hand rankings (weakest to strongest): ['High Card', 'One Pair', 'Two Pair', 'Three of a Kind', 'Straight']...


In [74]:
# Main simulation function

def simulate_one_round() -> Tuple[str, HandType, int]:
    """
    Run one complete round of the poker simulation.
    
    Returns:
        Tuple of (winner_name, winning_hand_type, points_earned)
    """
    print("STARTING ROUND SIMULATION")
    print("=" * 50)

    print("Adding custom decks to the main deck...")
    print(f"Custom decks added: Maddness, DarkSoul, Proletariat...")
    
    # Initialize game state
    current_real_deck = real_deck.copy()
    # add custom decks to the current real deck
    current_real_deck.extend(custom_decks[DeckType.Madness])
    current_real_deck.extend(custom_decks[DeckType.DarkSoul])
    current_real_deck.extend(custom_decks[DeckType.Proletariat])
    random.shuffle(current_real_deck)
    current_mod_deck = mod_deck.copy()
    random.shuffle(current_mod_deck)

    print(f"Deck shuffled: {len(current_real_deck)} cards in play")
    
    # Track player data
    player_hands: Dict[str, List[CardValue]] = {}
    player_mods: Dict[str, List[ModCard]] = {}
    
    print("\nPHASE 1: DEALING HANDS")
    print("-" * 30)
    
    # Deal cards to each player
    for player in players:
        # Deal real cards (5-8 cards)
        num_real = min(random.randint(5, 8), len(current_real_deck))
        real_cards = [current_real_deck.pop() for _ in range(num_real)]
        
        # Deal mod cards to fill hand
        hand_size = 8
        num_mod = max(0, hand_size - len(real_cards))
        num_mod = min(num_mod, len(current_mod_deck))
        mod_cards = [current_mod_deck.pop() for _ in range(num_mod)] if num_mod > 0 else []
        
        player_hands[player.name] = real_cards
        player_mods[player.name] = mod_cards
        
        print(f"{player.name} ({player.char_class}):")
        print(f"  Real cards: {', '.join(real_cards[:8])}{'...' if len(real_cards) > 5 else ''}")
        print(f"  Mod cards: {[mod.name for mod in mod_cards]}")
    
    print("\nPHASE 2: APPLYING MOD EFFECTS")
    print("-" * 30)
    
    # Apply mod effects
    for player in players:
        for mod in player_mods[player.name]:
            if mod.effect == ModCardEffects.Draw1 and current_real_deck:
                drawn = current_real_deck.pop()
                player_hands[player.name].append(drawn)
                print(f"  -> {player.name} draws {drawn} (Extra Draw)")
            elif mod.effect == ModCardEffects.SneakySwap:
                # Simplified swap
                other_players = [p for p in players if p.name != player.name]
                if other_players and player_hands[player.name]:
                    target = random.choice(other_players)
                    if player_hands[target.name]:
                        my_card = player_hands[player.name].pop()
                        their_card = player_hands[target.name].pop()
                        player_hands[player.name].append(their_card)
                        player_hands[target.name].append(my_card)
                        print(f"  -> {player.name} swaps {my_card} <-> {their_card} with {target.name}")
    
    print("\nPHASE 3: CHARACTER ABILITIES")
    print("-" * 30)
    
    # Apply character abilities
    for player in players:
        if player.char_class == CharacterClass.Rogue:
            # Steal up to 2 cards
            for steal_num in range(2):
                targets = [p for p in players if p.name != player.name and player_hands[p.name]]
                if targets:
                    target = random.choice(targets)
                    stolen_card = player_hands[target.name].pop()
                    player_hands[player.name].append(stolen_card)
                    print(f"  -> {player.name} (Rogue) steals {stolen_card} from {target.name}")
        elif player.char_class == CharacterClass.Warrior:
            # Reveal one opponent card
            targets = [p for p in players if p.name != player.name and player_hands[p.name]]
            if targets:
                target = random.choice(targets)
                revealed = random.choice(player_hands[target.name])
                print(f"  -> {player.name} (Warrior) reveals {target.name}'s {revealed}")
    
    print("\nPHASE 4: HAND EVALUATION")
    print("-" * 30)
    
    # Evaluate best hands
    player_best_hands: Dict[str, HandType] = {}
    for player in players:
        cards = player_hands[player.name]
        best_hand = cast(HandType, "High Card")
        
        if len(cards) >= 5:
            # Try all 5-card combinations
            for combo in combinations(cards, 5):
                hand_type = evaluate_standard_poker(list(combo))
                if get_hand_rank(hand_type) > get_hand_rank(best_hand):
                    best_hand = hand_type
        
        player_best_hands[player.name] = best_hand
        print(f"  {player.name}: {best_hand}")
    
    # Determine winner
    winner = max(player_best_hands, key=lambda x: get_hand_rank(player_best_hands[x]))
    winner_hand = player_best_hands[winner]
    
    print(f"\nWINNER: {winner} with {winner_hand}!")
    
    # Calculate points
    high_tier_hands = {"Flush Five", "Five of a Kind"}
    points_earned = 2 if winner_hand in high_tier_hands else 1
    chips_earned = 0
    
    for mod in player_mods[winner]:
        if mod.effect == ModCardEffects.RoyalDividend:
            chips_earned += 3
            print(f"  {winner} gains +3 chips from {mod.name}")
    
    print(f"\nROUND RESULTS:")
    print(f"  Winner: {winner} (+{points_earned} points, +{chips_earned} bonus chips)")
    print(f"  Winning Hand: {winner_hand}")
    
    return winner, winner_hand, points_earned

print("Simulation function ready!")
print("Run the next cell to simulate a poker round!")

Simulation function ready!
Run the next cell to simulate a poker round!


In [78]:
# Test the improved simulation function

print("🎲 TESTING IMPROVED CARD DEALING SYSTEM")
print("=" * 50)

# Run one round to see the improvements
winner, hand, points = simulate_one_round()

print(f"\n🏆 SIMULATION COMPLETE!")
print(f"Winner: {winner} with {hand} ({points} points)")

🎲 TESTING IMPROVED CARD DEALING SYSTEM
STARTING ROUND SIMULATION
Adding custom decks to the main deck...
Custom decks added: Maddness, DarkSoul, Proletariat...
Deck shuffled: 178 cards in play

PHASE 1: DEALING HANDS
------------------------------
Scruffy McMuffins (CharacterClass.Rogue):
  Real cards: 7C, JOKER, JOKER, 8H, 5D
  Mod cards: ['Sneaky Swap', 'Royal Dividend', 'Extra Draw']
Patches (CharacterClass.Jester):
  Real cards: 10S, 3S, KC, 10D, 10D, AC, JS...
  Mod cards: []
Sir Whiskers (CharacterClass.Warrior):
  Real cards: JC, 6C, 5H, 5C, 10C, 7C...
  Mod cards: []

PHASE 2: APPLYING MOD EFFECTS
------------------------------
  -> Scruffy McMuffins swaps 5D <-> 7C with Sir Whiskers
  -> Scruffy McMuffins draws 9S (Extra Draw)

PHASE 3: CHARACTER ABILITIES
------------------------------
  -> Scruffy McMuffins (Rogue) steals JS from Patches
  -> Scruffy McMuffins (Rogue) steals AC from Patches
  -> Sir Whiskers (Warrior) reveals Patches's 10D

PHASE 4: HAND EVALUATION
---------

In [ ]:
# Improved card dealing system with configurable parameters

# Game configuration constants
MIN_REAL_CARDS = 1
MAX_REAL_CARDS = 8
TOTAL_HAND_SIZE = 8
MAX_DISPLAY_CARDS = 6

def deal_cards_to_players(
    players: List[PlayerCharacterCard], 
    real_deck: List[CardValue], 
    mod_deck: List[ModCard]
) -> Tuple[Dict[str, List[CardValue]], Dict[str, List[ModCard]]]:
    """
    Deal cards to all players with balanced distribution.
    
    Args:
        players: List of player character cards
        real_deck: Shuffled real card deck 
        mod_deck: Shuffled mod card deck
        
    Returns:
        Tuple of (player_hands, player_mods) dictionaries
    """
    player_hands: Dict[str, List[CardValue]] = {}
    player_mods: Dict[str, List[ModCard]] = {}
    
    # Pre-determine card distribution for fairness
    card_distributions = []
    for player in players:
        # Each player gets between MIN_REAL_CARDS and MAX_REAL_CARDS real cards
        num_real = random.randint(MIN_REAL_CARDS, MAX_REAL_CARDS)
        num_mod = TOTAL_HAND_SIZE - num_real
        card_distributions.append((player.name, num_real, num_mod))
    
    print(f"Card distribution planned:")
    for name, num_real, num_mod in card_distributions:
        print(f"  {name}: {num_real} real + {num_mod} mod = {num_real + num_mod} total")
    
    # Deal cards according to planned distribution
    for player_name, num_real, num_mod in card_distributions:
        # Safety check for deck availability
        if len(real_deck) < num_real:
            print(f"⚠️  Warning: Not enough real cards for {player_name} ({len(real_deck)} available, {num_real} needed)")
            num_real = len(real_deck)
            num_mod = TOTAL_HAND_SIZE - num_real  # Adjust mod cards to maintain hand size
        
        if len(mod_deck) < num_mod:
            print(f"⚠️  Warning: Not enough mod cards for {player_name} ({len(mod_deck)} available, {num_mod} needed)")
            num_mod = len(mod_deck)
        
        # Deal real cards
        real_cards = [real_deck.pop() for _ in range(num_real)]
        
        # Deal mod cards
        mod_cards = [mod_deck.pop() for _ in range(num_mod)]
        
        player_hands[player_name] = real_cards
        player_mods[player_name] = mod_cards
        
        # Enhanced display with better formatting
        real_cards_display = real_cards[:MAX_DISPLAY_CARDS]
        real_cards_str = ', '.join(real_cards_display)
        if len(real_cards) > MAX_DISPLAY_CARDS:
            real_cards_str += f" ... (+{len(real_cards) - MAX_DISPLAY_CARDS} more)"
        
        mod_cards_str = [mod.name for mod in mod_cards]
        
        print(f"✅ {player_name}:")
        print(f"    Real cards ({len(real_cards)}): {real_cards_str}")
        print(f"    Mod cards ({len(mod_cards)}): {mod_cards_str}")
        print(f"    Total hand size: {len(real_cards) + len(mod_cards)}")
    
    return player_hands, player_mods

print("Enhanced card dealing system loaded!")
print(f"Configuration: {MIN_REAL_CARDS}-{MAX_REAL_CARDS} real cards, total hand size {TOTAL_HAND_SIZE}")

Enhanced card dealing system loaded!
Configuration: 5-7 real cards, total hand size 8


In [6]:
# Run multiple rounds for a tournament

def run_tournament(num_rounds: int = 10) -> Dict[str, int]:
    """Run a multi-round tournament."""
    print(f"STARTING {num_rounds}-ROUND TOURNAMENT")
    print("=" * 60)
    
    # Track player statistics
    player_stats = {player.name: {"wins": 0, "points": 0, "hands": []} for player in players}
    
    # Run all rounds
    for round_num in range(1, num_rounds + 1):
        print(f"\nROUND {round_num}/{num_rounds}")
        print("-" * 40)
        
        winner, hand_type, points = simulate_one_round()
        
        # Update statistics
        player_stats[winner]["wins"] += 1
        player_stats[winner]["points"] += points
        player_stats[winner]["hands"].append(hand_type)
        
        print(f"Round {round_num} Winner: {winner} ({hand_type})")
    
    print("\nTOURNAMENT RESULTS")
    print("=" * 60)
    
    # Sort players by points
    ranked_players = sorted(player_stats.items(), key=lambda x: x[1]["points"], reverse=True)
    
    # Display final standings
    medals = ["1st", "2nd", "3rd"]
    for rank, (name, stats) in enumerate(ranked_players):
        medal = medals[rank] if rank < 3 else f"{rank+1}th"
        print(f"{medal} {name}")
        print(f"    Points: {stats['points']} | Wins: {stats['wins']}/{num_rounds}")
        print(f"    Hands: {', '.join(stats['hands'])}")
        print()
    
    champion = ranked_players[0][0]
    print(f"TOURNAMENT CHAMPION: {champion}!")
    
    return {name: stats["points"] for name, stats in player_stats.items()}

# Run a 10-round tournament
print("Running a 10 round tournament...\n")
final_scores = run_tournament(10)

print(f"\nFinal Scores: {final_scores}")

Running a 10 round tournament...

STARTING 10-ROUND TOURNAMENT

ROUND 1/10
----------------------------------------
STARTING ROUND SIMULATION
Adding custom decks to the main deck...
Custom decks added: Maddness, DarkSoul, Proletariat...
Deck shuffled: 178 cards in play

PHASE 1: DEALING HANDS
------------------------------
Scruffy McMuffins (CharacterClass.Rogue):
  Real cards: JC, 9H, 8D, 3S, 8S
  Mod cards: ['Sneaky Swap', 'Royal Dividend', 'Extra Draw']
Patches (CharacterClass.Jester):
  Real cards: KC, 4S, 9D, AH, 4D, 3C, 6S...
  Mod cards: []
Sir Whiskers (CharacterClass.Warrior):
  Real cards: 7C, 6C, 8H, 2S, 10H
  Mod cards: []

PHASE 2: APPLYING MOD EFFECTS
------------------------------
  -> Scruffy McMuffins swaps 8S <-> 10H with Sir Whiskers
  -> Scruffy McMuffins draws 4C (Extra Draw)

PHASE 3: CHARACTER ABILITIES
------------------------------
  -> Scruffy McMuffins (Rogue) steals 8S from Sir Whiskers
  -> Scruffy McMuffins (Rogue) steals 2S from Sir Whiskers
  -> Sir Whis

In [7]:
# Import visualization libraries
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

print("Visualization libraries loaded!")
print("Available chart types: Bar, Pie, Scatter, Heatmap, and more!")

Visualization libraries loaded!
Available chart types: Bar, Pie, Scatter, Heatmap, and more!


In [8]:
# Visualization functions
def create_tournament_dashboard(tournament_data: Dict):
    """Create a comprehensive tournament dashboard with multiple visualizations."""
    
    player_stats = tournament_data["player_stats"]
    round_results = tournament_data["round_results"]
    hand_frequency = tournament_data["hand_frequency"]
    num_rounds = tournament_data["num_rounds"]
    
    # Prepare data for visualizations
    player_names = list(player_stats.keys())
    player_points = [player_stats[name]["points"] for name in player_names]
    player_wins = [player_stats[name]["wins"] for name in player_names]
    player_classes = [player_stats[name]["character_class"] for name in player_names]
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            "Final Tournament Standings", 
            "Hand Type Distribution",
            "Wins by Character Class",
            "Round-by-Round Performance"
        ),
        specs=[[{"type": "bar"}, {"type": "pie"}],
               [{"type": "bar"}, {"type": "scatter"}]]
    )
    
    # 1. Tournament Standings (Bar Chart)
    fig.add_trace(
        go.Bar(
            x=player_names,
            y=player_points,
            name="Points",
            text=[f"{pts} pts<br>{wins} wins" for pts, wins in zip(player_points, player_wins)],
            textposition="auto",
            marker_color=px.colors.qualitative.Set3[:len(player_names)]
        ),
        row=1, col=1
    )
    
    # 2. Hand Distribution (Pie Chart)
    fig.add_trace(
        go.Pie(
            labels=list(hand_frequency.keys()),
            values=list(hand_frequency.values()),
            name="Hand Types"
        ),
        row=1, col=2
    )
    
    # 3. Wins by Character Class (Bar Chart)
    class_wins = {}
    for name in player_names:
        char_class = player_stats[name]["character_class"]
        wins = player_stats[name]["wins"]
        class_wins[char_class] = class_wins.get(char_class, 0) + wins
    
    fig.add_trace(
        go.Bar(
            x=list(class_wins.keys()),
            y=list(class_wins.values()),
            name="Class Wins",
            marker_color=px.colors.qualitative.Pastel1[:len(class_wins)]
        ),
        row=2, col=1
    )
    
    # 4. Round Performance (Scatter Plot)
    for i, name in enumerate(player_names):
        rounds = list(range(1, num_rounds + 1))
        cumulative_points = []
        points_so_far = 0
        
        for round_result in round_results:
            if round_result["winner"] == name:
                points_so_far += round_result["points"]
            cumulative_points.append(points_so_far)
        
        fig.add_trace(
            go.Scatter(
                x=rounds,
                y=cumulative_points,
                mode='lines+markers',
                name=f"{name}",
                line=dict(width=3),
                marker=dict(size=8)
            ),
            row=2, col=2
        )
    
    # Update layout
    fig.update_layout(
        height=800,
        title_text=f"🎲 Poker Tournament Dashboard - {num_rounds} Rounds",
        title_x=0.5,
        showlegend=True,
        template="plotly_white"
    )
    
    # Update axes labels
    fig.update_xaxes(title_text="Players", row=1, col=1)
    fig.update_yaxes(title_text="Points", row=1, col=1)
    fig.update_xaxes(title_text="Character Class", row=2, col=1)
    fig.update_yaxes(title_text="Total Wins", row=2, col=1)
    fig.update_xaxes(title_text="Round Number", row=2, col=2)
    fig.update_yaxes(title_text="Cumulative Points", row=2, col=2)
    
    return fig

def create_character_analysis(tournament_data: Dict):
    """Create detailed character class performance analysis."""
    player_stats = tournament_data["player_stats"]
    
    # Analyze performance by character class
    class_analysis = {}
    for name, stats in player_stats.items():
        char_class = stats["character_class"]
        if char_class not in class_analysis:
            class_analysis[char_class] = {
                "players": [],
                "total_points": 0,
                "total_wins": 0,
                "hands_won_with": []
            }
        
        class_analysis[char_class]["players"].append(name)
        class_analysis[char_class]["total_points"] += stats["points"]
        class_analysis[char_class]["total_wins"] += stats["wins"]
        class_analysis[char_class]["hands_won_with"].extend(stats["hands"])
    
    # Create visualization
    classes = list(class_analysis.keys())
    avg_points = [class_analysis[cls]["total_points"] / len(class_analysis[cls]["players"]) 
                  for cls in classes]
    total_wins = [class_analysis[cls]["total_wins"] for cls in classes]
    
    fig = go.Figure()
    
    # Add bars for average points
    fig.add_trace(
        go.Bar(
            name="Avg Points per Player",
            x=classes,
            y=avg_points,
            yaxis="y",
            offsetgroup=1,
            marker_color=px.colors.qualitative.Set2
        )
    )
    
    # Add bars for total wins
    fig.add_trace(
        go.Bar(
            name="Total Wins",
            x=classes,
            y=total_wins,
            yaxis="y2",
            offsetgroup=2,
            marker_color=px.colors.qualitative.Dark2
        )
    )
    
    fig.update_layout(
        title="⚔️ Character Class Performance Analysis",
        xaxis=dict(title="Character Class"),
        yaxis=dict(title="Average Points per Player", side="left"),
        yaxis2=dict(title="Total Wins", side="right", overlaying="y"),
        template="plotly_white",
        height=500
    )
    
    return fig

print("Visualization functions created!")
print("Ready to generate beautiful charts from tournament data!")

Visualization functions created!
Ready to generate beautiful charts from tournament data!


In [9]:
# Enhanced tournament function with detailed tracking
def run_tournament_with_stats(num_rounds: int = 5) -> Dict:
    """Run a tournament with comprehensive statistics tracking."""
    print(f"STARTING {num_rounds}-ROUND TOURNAMENT WITH VISUALIZATION")
    print("=" * 60)
    
    # Initialize comprehensive tracking
    player_stats = {
        player.name: {
            "wins": 0, 
            "points": 0, 
            "hands": [], 
            "character_class": player.char_class.value,
            "abilities_used": 0,
            "mod_cards_played": 0,
            "round_placements": []
        } 
        for player in players
    }
    
    round_results = []
    hand_frequency = {}
    
    # Run all rounds
    for round_num in range(1, num_rounds + 1):
        print(f"\nROUND {round_num}/{num_rounds}")
        print("-" * 40)
        
        winner, hand_type, points = simulate_one_round()
        
        # Track hand frequency
        hand_frequency[hand_type] = hand_frequency.get(hand_type, 0) + 1
        
        # Update player statistics
        player_stats[winner]["wins"] += 1
        player_stats[winner]["points"] += points
        player_stats[winner]["hands"].append(hand_type)
        
        # Record round result
        round_results.append({
            "round": round_num,
            "winner": winner,
            "hand_type": hand_type,
            "points": points
        })
        
        # Track placements for all players this round
        round_scores = {}
        for player_name in player_stats.keys():
            if player_name == winner:
                round_scores[player_name] = get_hand_rank(hand_type)
            else:
                # Simulate other players' hands for placement
                round_scores[player_name] = random.randint(0, get_hand_rank(hand_type) - 1)
        
        # Sort by score and assign placements
        sorted_players = sorted(round_scores.items(), key=lambda x: x[1], reverse=True)
        for placement, (player_name, _) in enumerate(sorted_players, 1):
            player_stats[player_name]["round_placements"].append(placement)
        
        print(f"Round {round_num} Winner: {winner} ({hand_type})")
    
    print("\nTOURNAMENT COMPLETED - GENERATING VISUALIZATIONS...")
    
    return {
        "player_stats": player_stats,
        "round_results": round_results,
        "hand_frequency": hand_frequency,
        "num_rounds": num_rounds
    }

print("Enhanced tournament function ready!")
print("This version tracks detailed statistics for rich visualizations.")

Enhanced tournament function ready!
This version tracks detailed statistics for rich visualizations.


In [10]:
# Run enhanced tournament with visualizations!

print("🎲 RUNNING ENHANCED TOURNAMENT WITH VISUALIZATIONS")
print("=" * 60)

# Run tournament with detailed tracking
tournament_data = run_tournament_with_stats(5)

print(f"\n📊 GENERATING INTERACTIVE DASHBOARDS...")
print("-" * 40)

# Create and display main dashboard
dashboard = create_tournament_dashboard(tournament_data)
dashboard.show()

print("✅ Main tournament dashboard displayed above!")

🎲 RUNNING ENHANCED TOURNAMENT WITH VISUALIZATIONS
STARTING 5-ROUND TOURNAMENT WITH VISUALIZATION

ROUND 1/5
----------------------------------------
STARTING ROUND SIMULATION
Adding custom decks to the main deck...
Custom decks added: Maddness, DarkSoul, Proletariat...
Deck shuffled: 178 cards in play

PHASE 1: DEALING HANDS
------------------------------
Scruffy McMuffins (CharacterClass.Rogue):
  Real cards: 8H, 5S, KC, 5C, QS, JD, KS...
  Mod cards: ['Royal Dividend']
Patches (CharacterClass.Jester):
  Real cards: 4C, 7S, 8H, JS, 6S, 5C...
  Mod cards: ['Sneaky Swap', 'Extra Draw']
Sir Whiskers (CharacterClass.Warrior):
  Real cards: 4H, 4S, JD, 5D, 6H, 3S, 10C...
  Mod cards: []

PHASE 2: APPLYING MOD EFFECTS
------------------------------
  -> Patches swaps 5C <-> 10C with Sir Whiskers
  -> Patches draws 9D (Extra Draw)

PHASE 3: CHARACTER ABILITIES
------------------------------
  -> Scruffy McMuffins (Rogue) steals 5C from Sir Whiskers
  -> Scruffy McMuffins (Rogue) steals 9D fr

✅ Main tournament dashboard displayed above!


In [11]:
# Hand rarity and exotic hand analysis

def create_hand_rarity_analysis(tournament_data: Dict):
    """Analyze the rarity and frequency of different hand types."""
    hand_frequency = tournament_data["hand_frequency"]
    
    # Categorize hands by rarity
    common_hands = ["High Card", "One Pair", "Two Pair"]
    uncommon_hands = ["Three of a Kind", "Straight", "Flush"]
    rare_hands = ["Full House", "Four of a Kind", "Straight Flush"]
    exotic_hands = ["Royal Flush", "Flush Four", "Sandwich Hand", "Odd Straight", 
                   "Even Straight", "Skipping Straight", "Rainbow Straight", 
                   "Flush House", "Five of a Kind", "Flush Five"]
    
    categories = {
        "Common": sum(hand_frequency.get(hand, 0) for hand in common_hands),
        "Uncommon": sum(hand_frequency.get(hand, 0) for hand in uncommon_hands),
        "Rare": sum(hand_frequency.get(hand, 0) for hand in rare_hands),
        "Exotic": sum(hand_frequency.get(hand, 0) for hand in exotic_hands)
    }
    
    # Create donut chart
    fig = go.Figure(data=[go.Pie(
        labels=list(categories.keys()),
        values=list(categories.values()),
        hole=0.4,
        marker_colors=px.colors.qualitative.Set3
    )])
    
    fig.update_layout(
        title="🃏 Hand Rarity Distribution",
        template="plotly_white",
        height=400
    )
    
    # Add center text
    total_hands = sum(categories.values())
    exotic_pct = (categories["Exotic"] / total_hands * 100) if total_hands > 0 else 0
    
    fig.add_annotation(
        text=f"Exotic Hands<br>{exotic_pct:.1f}%",
        x=0.5, y=0.5,
        font_size=16,
        showarrow=False
    )
    
    return fig

# Create and display hand rarity analysis
print("🃏 ANALYZING HAND RARITY DISTRIBUTION")
print("-" * 40)

hand_rarity_chart = create_hand_rarity_analysis(tournament_data)
hand_rarity_chart.show()

print("✅ Hand rarity analysis displayed above!")

# Show detailed breakdown
print("\nDetailed Hand Frequency:")
for hand_type, count in sorted(tournament_data["hand_frequency"].items(), 
                              key=lambda x: x[1], reverse=True):
    percentage = (count / tournament_data["num_rounds"]) * 100
    print(f"  {hand_type}: {count} times ({percentage:.1f}%)")

🃏 ANALYZING HAND RARITY DISTRIBUTION
----------------------------------------


✅ Hand rarity analysis displayed above!

Detailed Hand Frequency:
  Four of a Kind: 4 times (80.0%)
  Full House: 1 times (20.0%)
